# Segunda Avaliação Parcial

Neste módulo trabalhamos a visualização de dados, assim como as técnicas para acessar estes dados da internet. As técnicas de web scraping que estudamos, usando __Requests__, __Beautiful Soup__ e __Selenium__, permitem implementar aplicações que processam informação diretamente da rede.  

Desde a Instrução prática 7, por exemplo, estamos trabalhando com dados que baixamos do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos).  

Aprendemos também como implementar uma aplicação com GUI utilizando o __TK Inter__. 

Neste contexto vamos implementar uma aplicação que, a traves de uma GUI, permita gerar gráficos de precipitações e temperaturas medias mensais de um determinado ano, para uma estação meteorológica. A atividade deve ser organizada da seguinte forma.

## Exercício 1

__Primeiro protótipo__: Desenvolva uma interface gráfica que pegue, em tempo real, as informações sobre os dados de quais anos estão disponíveis no site do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). Os anos disponíveis devem ser apresentados utilizando um widget apropriado que permita selecionar um, e apenas um ano. Se não tiver conexão a Internet ou se o site não estiver disponível o aplicativo deve mostrar uma mensagem de erro na interface.


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import requests
from bs4 import BeautifulSoup

def obter_dados():
    link = "https://portal.inmet.gov.br/dadoshistoricos"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    try:
        requisicao = requests.get(link, headers=headers)
        requisicao.raise_for_status()  # Verifica se a requisição foi bem sucedida
    except requests.exceptions.RequestException as e:
        messagebox.showerror("Erro", f"Erro na requisição: {str(e)}")
        return None

    site = BeautifulSoup(requisicao.text, 'html.parser')
    anos = site.find_all('article', class_='post-preview')
    ano = [i.find('a').string for i in anos]
    return ano

def mostrar_erro():
    messagebox.showerror("Erro", "Erro na conexão com o site. Verifique sua conexão com a internet.")

def mostrar_ano_selecionado(var_ano):
    selected_ano = var_ano.get()
    messagebox.showinfo("Ano Selecionado", f"Ano selecionado: {selected_ano}")

def main():
    root = tk.Tk()
    root.title('Dados Meteorológicos')
    root.geometry("400x250")

    anos = obter_dados()

    if anos:
        label_ano = tk.Label(root, text='Selecione o Ano:', font=('Arial', 15))
        label_ano.pack()

        var_ano = tk.StringVar()
        combobox_ano = ttk.Combobox(root, textvariable=var_ano, values=anos, font=('Arial', 15))
        combobox_ano.pack()

        button_mostrar_ano = ttk.Button(root, text="Mostrar Ano Selecionado", command=lambda: mostrar_ano_selecionado(var_ano))
        button_mostrar_ano.pack()
    else:
        mostrar_erro()

    style = ttk.Style()
    style.configure('TButton', font=('Arial', 15))

    root.mainloop()

if __name__ == "__main__":
    main()


## Exercício 2

__Segundo protótipo__: Com base no ano escolhido sua aplicação deve baixar o arquivo compactado, com os dados de todas as estações, descompactar ele numa pasta temporária e mostrar em um outro widget a lista de estações disponíveis para selecionar uma e apenas uma delas. O widget do protótipo anterior deve ficar disponível para que o usuário possa trocar o ano se desejar. Nesse caso um novo arquivo será baixado e descompactado e as estações disponíveis exibidas para seleção.

In [5]:
import os
from tkinter import messagebox
import zipfile
import IPython

import requests


#IPython
def obter_estacoes(ano_selecionado):
  # Formatando o nome do arquivo
  nome_arquivo = ano_selecionado.replace(' ', '_').replace('(', '').replace(')', '')

  # Construindo o link para download do arquivo zip
  link = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano_selecionado}.zip"

  # Cria um diretório temporário para armazenar o arquivo zip
  temp_dir = 'temp_zip_folder'
  os.makedirs(temp_dir, exist_ok=True)
  print(f'Diretório temporário criado em: {temp_dir}')

  # Define o caminho completo para o arquivo zip
  zip_file_path = os.path.join(temp_dir, f'{nome_arquivo}.zip')

  # Tenta baixar o arquivo zip
  try:
    with requests.get(link, stream=True) as response:
      response.raise_for_status()
      with open(zip_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
          if chunk:
            file.write(chunk)

    # Tenta descompactar o arquivo zip
    try:
      with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Cria um subdiretório se o ano for maior que 2019
        if int(ano_selecionado) > 2019:
          temp_dir = os.path.join(temp_dir, ano_selecionado)
          os.makedirs(temp_dir, exist_ok=True)
        zip_ref.extractall(temp_dir)

        # Retorna a lista de estações disponíveis
        if int(ano_selecionado) <= 2019:
          estacoes = os.path.join(temp_dir, ano_selecionado)
          estacoes = os.listdir(estacoes)
        else:
          estacoes = os.listdir(temp_dir)
        return estacoes
    except zipfile.BadZipFile:
      messagebox.showerror("Erro", f"Erro: O arquivo baixado para o ano {ano_selecionado} não é um arquivo zip válido.")
      return None
  except requests.exceptions.RequestException as e:
    messagebox.showerror("Erro", f"Erro na requisição: {e}")
    return None

# Exercício 3

__Terceiro protótipo__: Com base na estação selecionada gere e exiba dentro da aplicação os gráficos de precipitações e temperaturas medias mensais para a estação selecionada. A interface deve permitir que o usuário troque a estação selecionada a qualquer momento, assim como foi feito co o ano, gerando novos gráficos.   

In [ ]:

# Função para gerar gráficos
import tkinter
from tkinter import Tk, TkVersion, ttk
from turtle import pd

from matplotlib import pyplot as plt
from meteorologico.meteo_package import mostrar_erro, obter_dados, obter_dados_estacao


def gerar_graficos(ano_selecionado, estacao_selecionada, janela):
    dados_estacao = obter_dados_estacao(ano_selecionado, estacao_selecionada)
    dados_estacao['DATA (YYYY-MM-DD)'] = pd.to_datetime(dados_estacao['DATA (YYYY-MM-DD)'])
    dados_estacao['month'] = dados_estacao['DATA (YYYY-MM-DD)'].dt.month

    fig, (ax, ax2) = plt.subplots(2, 1, figsize=(8, 6), dpi=100)

    index = pd.DatetimeIndex(dados_estacao.iloc[:, 0], name='Data')
    dados_estacao.drop(columns=dados_estacao.columns[0], axis=1, inplace=True)
    dados_estacao.set_index(index, inplace=True)

    dados_estacao['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'] = dados_estacao['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].apply(
        lambda x: None if x < -14 else x)
    temp = dados_estacao.groupby(dados_estacao['month'])['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].mean()
    temp.plot(ax=ax, title="Temperatura Média Mensal", ylabel="Temperatura (°C)", color='blue')
    ax.set_xlabel("Mês", fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(["Temperatura Média"], loc="upper right")

    dados_estacao['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = dados_estacao['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].apply(
        lambda x: None if x < 0 else x)
    prec = dados_estacao.groupby(dados_estacao['month'])['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].sum()
    prec.plot(ax=ax2, title="Precipitação Mensal", ylabel="Precipitação (mm)", color='green')
    ax2.set_xlabel("Mês", fontsize=12)
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.legend(["Precipitação"], loc="upper right")

    fig.subplots_adjust(wspace=0.5, hspace=0.5)

    canvas = FigureCanvasTkAgg(fig, master=janela)
    canvas.get_tk_widget().pack()

def main():
    root = tkinter.Tk()
    root.title('Dados Meteorológicos')
    root.geometry("800x600")

    dados_site = obter_dados()

    if dados_site:
        ano, link = dados_site

        label_ano = tkinter.Tk.Label(root, text='Selecione o Ano:', font=('Arial', 15))
        label_ano.pack()

        var_ano = tkinter.Tk.StringVar()
        combobox_ano = ttk.Combobox(root, textvariable=var_ano, values=ano, font=('Arial', 15))
        combobox_ano.pack()

        def obter_estacoes_selecionar_ano():
            ano_selecionado = var_ano.get()
            if ano_selecionado:
                estacoes = obter_estacoes(ano_selecionado[4:8])
                if estacoes:
                    var_estacoes.set('')
                    combobox_estacoes['values'] = estacoes
                else:
                    mostrar_erro()

        botao_obter_estacoes = ttk.Button(root, text='Obter Estações', command=obter_estacoes_selecionar_ano, style='TButton')
        botao_obter_estacoes.pack()

        label_estacoes = Tk.Label(root, text='Selecione a Estação:', font=('Arial', 15))
        label_estacoes.pack()

        var_estacoes = TkVersion.StringVar()
        combobox_estacoes = ttk.Combobox(root, textvariable=var_estacoes, font=('Arial', 15))
        combobox_estacoes.pack()

        def gerar_graficos_estacao():
            ano_selecionado = var_ano.get()
            estacao_selecionada = var_estacoes.get()
            if ano_selecionado and estacao_selecionada:
                nova_janela = tk.Toplevel(root)
                nova_janela.title(f'Gráficos - {estacao_selecionada}')
                gerar_graficos(ano_selecionado, estacao_selecionada, nova_janela)
            else:
                messagebox.showerror("Erro", "Selecione ano e estação antes de gerar os gráficos.")

        botao_gerar_graficos = ttk.Button(root, text='Gerar Gráficos', command=gerar_graficos_estacao, style='TButton')
        botao_gerar_graficos.pack()

    else:
        mostrar_erro()

    style = ttk.Style()
    style.configure('TButton', font=('Arial', 15))

    root.mainloop()

if __name__ == "__main__":
    main()




## Respostas

Implemente sua aplicação como um pacote __Python__ e disponibilize neste __Notebook__ um explicação dos principais pontos da implementação. Utilize o comando mágico ``%run`` para executar o aplicativo desenvolvido.  Envie o __Notebook__ via __Moodle__ assim como o repositório com a implementação do aplicativo até o final do prazo. 

In [3]:
%run meteorologico/__main__.py
